In [3]:
import sys
import binascii
try:   
    from capstone import *
except:
    print("no se puede encontrar capstone, use pip install capstone")
    sys.exit()
    
CODE = open("./alu_test.bin", mode = 'rb').read()
md = Cs(CS_ARCH_MIPS, CS_MODE_MIPS32 + CS_MODE_BIG_ENDIAN)

for i in md.disasm(CODE, 0x10000000):   
    print("0x%x: \t%s \t%s\t%s" %(i.address, binascii.hexlify(i.bytes) , i.mnemonic, i.op_str))

0x10000000: 	3c080000 	lui	$t0, 0
0x10000004: 	25080880 	addiu	$t0, $t0, 0x880
0x10000008: 	3c090000 	lui	$t1, 0
0x1000000c: 	25290880 	addiu	$t1, $t1, 0x880
0x10000010: 	11090003 	beq	$t0, $t1, 0x10000020
0x10000014: 	25080004 	addiu	$t0, $t0, 4
0x10000018: 	08000004 	j	0x10000010
0x1000001c: 	ad00fffc 	sw	$zero, -4($t0)
0x10000020: 	00000821 	move	$at, $zero
0x10000024: 	00001021 	move	$v0, $zero
0x10000028: 	00001821 	move	$v1, $zero
0x1000002c: 	00002021 	move	$a0, $zero
0x10000030: 	00002821 	move	$a1, $zero
0x10000034: 	00003021 	move	$a2, $zero
0x10000038: 	00003821 	move	$a3, $zero
0x1000003c: 	00004021 	move	$t0, $zero
0x10000040: 	00004821 	move	$t1, $zero
0x10000044: 	00005021 	move	$t2, $zero
0x10000048: 	00005821 	move	$t3, $zero
0x1000004c: 	00006021 	move	$t4, $zero
0x10000050: 	00006821 	move	$t5, $zero
0x10000054: 	00007021 	move	$t6, $zero
0x10000058: 	00007821 	move	$t7, $zero
0x1000005c: 	00008021 	move	$s0, $zero
0x10000060: 	00008821 	move	$s1, $zero
0x10000064: 	

In [64]:
#segunda version
import binascii

registers = {
    '00000': '$zero','01000':'$t0','10000':'$s0','11000':'$t8',
    '00001': '$at'  ,'01001':'$t1','10001':'$s1','11001':'$t9',
    '00010': '$v0'  ,'01010':'$t2','10010':'$s2','11010':'$k0',
    '00011': '$v1'  ,'01011':'$t3','10011':'$s3','11011':'$k1',
    '00100': '$a0'  ,'01100':'$t4','10100':'$s4','11100':'$gp',
    '00101': '$a1'  ,'01101':'$t5','10101':'$s5','11101':'$sp',
    '00110': '$a2'  ,'01110':'$t6','10110':'$s6','11110':'$fp',
    '00111': '$a3'  ,'01111':'$t7','10111':'$s7','11111':'$ra',
}

# format \times opcode \times funct
operations = {
    ('R', '000000', '000000') :'nop',
    ('R', '000000', '100000') :'add',
    ('I', '001000', None)     :'addi',
    ('R', '000000', '100100') :'and',
    ('I', '000100', None)     :'beq',
    ('J', '000010', None)     :'j',
    ('J', '000011', None)     :'jal',
    ('R', '000000', '001000') :'jr',
    ('I', '100011', '100011') :'lw',
    ('R', '000000', '100101') :'or',
    ('R', '000000', '101010') :'slt',
    ('R', '000000', '000000') :'sll',
    ('I', '101011', None)     :'sw',
    ('R', '000000', '100010') :'sub',
}
        
bin_file = open("./alu_test.bin", mode = 'rb')

scale = 16

def instruction_generator(bin_file):
    while 1:
        try:
            yield bin(int(binascii.hexlify(bin_file.read(4)), scale))[2:].zfill(32)
        except:
            return

instructions = instruction_generator(bin_file = bin_file)

for ins in instructions:
    # si fuera del tipo R
    opcode = ins[0:6]
    rs = ins[6:11]
    rt = ins[11:16]
    rd = ins[16:21]
    shamt = ins[21:26]
    funct = ins[26:32]
    
    #si fuera del tipo I
    address = ins[16:32]
    
    # si fuera del tipo J
    jump = ins[11:32]
    
    try:
        print binascii.hexlify(ins), operations[("R", opcode, funct)], registers[rs], registers[rt], registers[rd]
    except:
        try:
            operations[("I", opcode, None)]
        except:
            try:
                operations[("J",opcode, None)]
            except:
                print "unknow"
        
    

3030313131313030303030303130303030303030303030303030303030303030 unknow
3030313030313031303030303130303030303030313030303130303030303030 unknow
3030313131313030303030303130303130303030303030303030303030303030 unknow
3030313030313031303031303130303130303030313030303130303030303030 unknow
3030303130303031303030303130303130303030303030303030303030303131 3030313030313031303030303130303030303030303030303030303030313030 unknow
3030303031303030303030303030303030303030303030303030303030313030 3130313031313031303030303030303031313131313131313131313131313030 3030303030303030303030303030303030303030313030303030313030303031 unknow
3030303030303030303030303030303030303031303030303030313030303031 unknow
3030303030303030303030303030303030303031313030303030313030303031 unknow
3030303030303030303030303030303030303130303030303030313030303031 unknow
3030303030303030303030303030303030303130313030303030313030303031 unknow
3030303030303030303030303030303030303131303030303030313030303031 unknow
3030303030303